<a href="https://colab.research.google.com/github/MarisaRies/covir/blob/main/COVIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Encryption
#!pip install cape_privacy

In [ ]:
#
#Datenbankaufbau
#

#Lade Bibliotheken
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import requests

#Pfad Definitionen
drive_path = "/content/drive"
material_path = f"{drive_path}/MyDrive/Studium/Material"
csv_file_path = f"{material_path}/csv_data/covid19/"

#Verbinden von GDrive
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

#Verbindung zu SQLite
def create_connection_memory():
  conn = None;
  try:
    #Verbindungsaufbau
    conn = sq.connect(':memory:')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Fehler #1 beim Verbinden zu SQLite", e)

#Lokale Definition der SQLite Verbindung
def create_connection_local(local_path):
  conn = None;
  try:
    #Verbindungsaufbau
    conn = sq.connect(local_path+'/Studium/covir.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Fehler #2 beim Verbinden zu SQLite", e) 
conn = create_connection_memory()

cur = conn.cursor()
print("Verbindung erfolgreich zu SQLite")

#Tabellenschema erzeugen
sql_create_source_data = requests.get('https://raw.githubusercontent.com/MarisaRies/covir/main/sql/db_create_statements.sql').text
cur.executescript(sql_create_source_data)
conn.commit()
print("Tabellenschema erfolgreich erstellt")

#CSV-Daten aus Google-Drive laden und in Datenbank laden
df_insert_pat = pd.read_csv(csv_file_path+'patients.csv')
#drop columns...

df_insert_pat.to_sql('tb_patients', conn, if_exists='append', index=False)

df_insert_pat = pd.read_csv(csv_file_path+'encounters.csv')
df_insert_pat.to_sql('tb_encounters', conn, if_exists='append', index=False)

df_insert_pat = pd.read_csv(csv_file_path+'conditions.csv')
df_insert_pat.to_sql('tb_conditions', conn, if_exists='append', index=False)

df_insert_pat = pd.read_csv(csv_file_path+'immunizations.csv')
df_insert_pat.to_sql('tb_immunizations', conn, if_exists='append', index=False)

conn.commit()

print("CSV-Daten erfolgreich geladen, Datenbank bereit")

Mounted at /content/drive
Verbindung erfolgreich zu SQLite
Tabellenschema erfolgreich erstellt


OperationalError: ignored